Die meist umkäpften Pixel im Canvas

In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9db973d1-7bec-4fc1-9563-eb7cc41c5d5d;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 70ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



Nun dataFrame nach Pixel gruppieren (Hab dafür eine entsprechende Methode ergänzt)

In [2]:
from src.features.feature_functions import group_dataframe_by_pixel, group_dataframe_by_columns

groupedFrame = group_dataframe_by_pixel(dataFrame)
groupedFrame.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(t): array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [3]:
groupedFrame.show(2)

+---+---+-----+---------------------+--------------------+-------------------------+
|  x|  y|count|collect_list(user_id)|     collect_list(t)|collect_list(pixel_color)|
+---+---+-----+---------------------+--------------------+-------------------------+
|  0|  0| 1568| [U7ZGIDmgxWHUBUud...|[3, 1612, 16, 161...|     [#7EED56, #FFFFFF...|
|359|564| 1524| [evgpuJ6pMHVzs8PR...|[1756, 2617, 1756...|     [#000000, #000000...|
+---+---+-----+---------------------+--------------------+-------------------------+
only showing top 2 rows



Spalten, die für die Visualisierung störend sind, entfernen.

In [5]:
from src.visualization.visuialization_functions import show_and_save
import matplotlib.pyplot as plt
import numpy as np

droppedFrame = groupedFrame.drop('collect_list(t)').drop('collect_list(user_id)').drop('collect_list(pixel_color)')
droppedFrame.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- count: long (nullable = false)



Die Top 10 der am meisten geänderten Pixel.

In [6]:
droppedFrame.show(10)

+----+----+-----+
|   x|   y|count|
+----+----+-----+
|   0|   0| 1568|
| 359| 564| 1524|
|1999|1999| 1240|
| 349| 564| 1226|
|   0|1999| 1032|
|1890| 353|  940|
|1852| 590|  926|
| 662| 222|  916|
|1058| 756|  914|
|   8|1267|  882|
+----+----+-----+
only showing top 10 rows



Das ganze noch visualisieren.

In [ ]:
datacollect = droppedFrame.collect()
#wie mache ich weiter???
fig,ax = plt.subplots()
for row in datacollect:
    pixelcount,usercount = list(row)
    ax.bar(pixelcount,usercount,color='#5050cc')
ax.set_xticks(np.arange(1,10,1))
ax.set_title('Die 10 am meisten umkämpften Pixel')
ax.set_ylabel('Anzahl der Änderungen')
ax.set_xlabel('Geänderte Pixel')
fig.tight_layout()
#plt.savefig('../reports/figures/meistumkäpften_pixel.jpg',bbox_inches = 'tight')
plt.show()